In [1]:
import os
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import wandb
from util import load_data
from dataset import TimeVAEdataset
from TimeVAE import TimeVAE
import argparse
import torch.nn as nn



In [2]:
params={'data_source': '29var',
         'batch_size': 20, 'device': 'cpu',
           'epochs': 100, 'latent_dim': 64, 'lr': 0.001,
             'break_data': False, 'break_size': 60,
               'leave_out_problematic_features': True, 
               'cutoff_data': True, 'feat_dim': 24,
                 'seq_len': 120, 'num_samples': 180, 
                 'trend_poly': 2, 'custom_seas': [(4, 8)], 
                 'reconstruction_wt': 3.0, 'hidden_layer_sizes': [50, 100, 200]}
if params["data_source"] == "29var":
    data = load_data(
        "../Data/PreProcessed/29var/df29.xlsx",
        break_to_smaller=params["break_data"],
        break_size=params["break_size"],
        leave_out_problematic_features=params["leave_out_problematic_features"],
        cutoff_data=params["cutoff_data"],
    )
elif params["data_source"] == "12var":
    data = load_data(
        "../Data/PreProcessed/12var/df12.xlsx",
        break_to_smaller=params["break_data"],
        break_size=params["break_size"],
        leave_out_problematic_features=params["leave_out_problematic_features"],
        cutoff_data=params["cutoff_data"],
    )
else:
    raise ValueError("Unsupported data source. Use 12var or 29var")
###
sequence_length = data.shape[1]
print("Parameter Sequence Length:", sequence_length)
number_of_features = data.shape[2]
print("Parameter Number of Features:", number_of_features)

dataset = TimeVAEdataset(data)
params["feat_dim"] = number_of_features
params["seq_len"] = sequence_length
params["num_samples"] = data.shape[0]
train_loader = torch.utils.data.DataLoader(
    dataset, batch_size=params["batch_size"], shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    dataset, batch_size=params["batch_size"], shuffle=True
)
params["trend_poly"] = 2
params["custom_seas"] = [(4, 8)]
params["reconstruction_wt"] = 3.0
params["hidden_layer_sizes"] = [50, 100, 200]

Number of samples: 180
Shape of _samples: (180, 120, 24)
Parameter Sequence Length: 120
Parameter Number of Features: 24


In [3]:
model=TimeVAE(params)

In [4]:
# for epoch in range(100):
#     for batch, data in enumerate(train_loader):
#         break

In [5]:
for batch in train_loader:
    print(batch)
    break

tensor([[[1., 1., 5.,  ..., 1., 4., 1.],
         [1., 1., 6.,  ..., 1., 6., 2.],
         [1., 1., 5.,  ..., 1., 5., 2.],
         ...,
         [1., 1., 6.,  ..., 1., 6., 1.],
         [1., 1., 6.,  ..., 1., 6., 1.],
         [1., 1., 6.,  ..., 1., 6., 1.]],

        [[1., 1., 7.,  ..., 2., 5., 5.],
         [1., 1., 6.,  ..., 1., 7., 1.],
         [1., 1., 7.,  ..., 1., 7., 3.],
         ...,
         [1., 1., 6.,  ..., 2., 5., 3.],
         [1., 1., 5.,  ..., 1., 5., 4.],
         [1., 1., 5.,  ..., 1., 4., 3.]],

        [[2., 1., 4.,  ..., 4., 4., 1.],
         [2., 1., 4.,  ..., 4., 6., 3.],
         [2., 1., 5.,  ..., 4., 5., 3.],
         ...,
         [2., 3., 4.,  ..., 3., 4., 2.],
         [3., 4., 4.,  ..., 4., 4., 1.],
         [3., 2., 4.,  ..., 3., 4., 2.]],

        ...,

        [[3., 3., 4.,  ..., 4., 5., 1.],
         [3., 3., 4.,  ..., 3., 5., 3.],
         [2., 2., 5.,  ..., 4., 6., 4.],
         ...,
         [3., 2., 3.,  ..., 3., 4., 1.],
         [2., 2., 4., 

In [6]:
batch.shape

torch.Size([20, 120, 24])

In [7]:
z1,z2,z3=model.encoder(batch)

In [8]:
print(z1.shape,z2.shape,z3.shape)

torch.Size([20, 64]) torch.Size([20, 64]) torch.Size([20, 64])


In [9]:
z=model._reparameterize(z1,z2)

In [10]:
z.shape

torch.Size([20, 64])

In [11]:
model.decoder(z)

Shape of outputs after level layer: torch.Size([20, 120, 24])
Shape of z before trend layer: torch.Size([20, 64])
Shape of trend_params after first dense layer: torch.Size([20, 48])
Shape of trend_params after second dense layer: torch.Size([20, 48])
Shape of trend_params after reshaping: torch.Size([20, 24, 2])
Shape of outputs after trend layer: torch.Size([20, 120, 24])


RuntimeError: 'weight' must be 2-D

In [ ]:
params['feat_dim']

In [ ]:
t1=nn.Linear(params['latent_dim'], params['latent_dim'] * params['trend_poly'])

In [ ]:
t1(z).shape

In [ ]:
t2=nn.Linear(params['latent_dim'] * params['trend_poly'], params['latent_dim'] * params['trend_poly'])

In [ ]:
q=t2(t1(z))
q2=q.view(-1, params['feat_dim'], params['trend_poly'])

In [ ]:
lin_space = torch.linspace(0, 1, params['seq_len'], device=z.device)
poly_space = torch.stack(
    [lin_space ** (p + 1) for p in range(params['trend_poly'])], dim=0
)

In [ ]:
trend_vals = torch.matmul(q2, poly_space)
trend_vals = trend_vals.permute(0, 2, 1)

In [ ]:
trend_vals